## Extract Poses from Amass Dataset

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [4]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
comp_device

device(type='cpu')

In [5]:
pwd

'/Users/aleksandrrazin/work/research/3d/HumanML3D'

In [7]:
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = './body_models/smplh/male/model.npz'
male_dmpl_path = './body_models/dmpls/male/model.npz'

female_bm_path = './body_models/smplh/female/model.npz'
female_dmpl_path = './body_models/dmpls/female/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).to(comp_device)
faces = c2c(male_bm.f)

female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path).to(comp_device)

In [9]:
os.listdir('./amass_data')

['LICENSE.txt', 'ACCAD', 'ACCAD.tar.bz2']

In [15]:
from pathlib import Path

paths = []
folders = []
dataset_names = []

for root, dirs, files in os.walk('./amass_data'):
    folders.append(root)
    parts = Path(root).parts
    dataset_name = parts[1] if len(parts) > 1 else None  # './amass_data/ACCAD' -> 'ACCAD'
    for name in files:
        if not name.endswith('.npz'):
            continue
        if dataset_name and dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))


In [16]:
save_root = './pose_data'
save_folders = [folder.replace('./amass_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

In [17]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]])
ex_fps = 20
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    
    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    if bdata['gender'] == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)

    bdata_poses = bdata['poses'][::down_sample,...]
    bdata_trans = bdata['trans'][::down_sample,...]
    body_parms = {
            'root_orient': torch.Tensor(bdata_poses[:, :3]).to(comp_device),
            'pose_body': torch.Tensor(bdata_poses[:, 3:66]).to(comp_device),
            'pose_hand': torch.Tensor(bdata_poses[:, 66:]).to(comp_device),
            'trans': torch.Tensor(bdata_trans).to(comp_device),
            'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=len(bdata_trans), axis=0)).to(comp_device),
        }
    
    with torch.no_grad():
        body = bm(**body_parms)
    pose_seq_np = body.Jtr.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    
    
    np.save(save_path, pose_seq_np_n)
    return fps

In [18]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [19]:
import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('./amass_data', './pose_data')
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)
        
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)

Processing: ACCAD: 100%|██████████| 252/252 [00:06<00:00, 41.76it/s]


Processed / All (fps 120): 252/252


The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions

In [20]:
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin

In [21]:
def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1
    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain = [1, 4, 7, 10, 13, 16, 18, 20]
    left_hand_chain = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]
    tmp = data[:, right_chain]
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain] = tmp
    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain]
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain] = tmp
    return data

In [22]:
index_path = './index.csv'
save_dir = './joints'
index_file = pd.read_csv(index_path)
total_amount = index_file.shape[0]
fps = 20

In [27]:
import os
os.makedirs('./joints', exist_ok=True)

In [28]:
index_file = index_file[index_file['source_path'].str.contains('ACCAD|humanact12')]
index_file = index_file.reset_index(drop=True)
total_amount = len(index_file)
index_file['source_path'] = index_file['source_path'].str.replace(
    './pose_data/humanact12/humanact12/',
    './pose_data/humanact12/',
    regex=False,
)

for i in tqdm(range(total_amount)):
    source_path = index_file.loc[i]['source_path']
    new_name = index_file.loc[i]['new_name']
    data = np.load(source_path)
    start_frame = index_file.loc[i]['start_frame']
    end_frame = index_file.loc[i]['end_frame']
    if 'humanact12' not in source_path:
        if 'Eyes_Japan_Dataset' in source_path:
            data = data[3*fps:]
        if 'MPI_HDM05' in source_path:
            data = data[3*fps:]
        if 'TotalCapture' in source_path:
            data = data[1*fps:]
        if 'MPI_Limits' in source_path:
            data = data[1*fps:]
        if 'Transitions_mocap' in source_path:
            data = data[int(0.5*fps):]
        data = data[start_frame:end_frame]
        data[..., 0] *= -1
    
    data_m = swap_left_right(data)
#     save_path = pjoin(save_dir, )
    np.save(pjoin(save_dir, new_name), data)
    np.save(pjoin(save_dir, 'M'+new_name), data_m)

  0%|          | 0/1468 [00:00<?, ?it/s]

100%|██████████| 1468/1468 [00:01<00:00, 1298.81it/s]


What a “dataset” is here
It’s a collection of motion recordings (people walking, jumping, kicking, etc.). Each dataset is a different lab or project that recorded motion capture.

AMASS

AMASS is a big collection that bundles many motion datasets together.
Names like ACCAD, CMU, KIT, HumanEva, etc. are just different sources inside AMASS.
HumanML3D is created by processing AMASS motions into a unified format.
HumanAct12

A smaller dataset of action clips (12 action types).
It’s already included in this repo, so you don’t download it separately.
KIT‑ML

A motion‑language dataset where motions have text descriptions.
This repo includes a processed KIT‑ML version for text‑to‑motion tasks.
What the “models” are

These are not ML models; they are body templates used to convert pose parameters into 3D joint positions.

SMPL+H

A 3D human body model that includes hands.
It defines a human skeleton and how a body mesh deforms when you move joints.
DMPL

A model for soft‑tissue dynamics (muscle/fat jiggle) over time.
It adds extra parameters for realistic motion deformation.
You need SMPL+H + DMPL because AMASS motions are stored as model parameters, not as explicit joint coordinates. The pipeline uses these models to decode the parameters into actual 3D joint positions.

In short

Datasets = motion recordings.
SMPL+H / DMPL = tools to turn those recordings (parameters) into 3D skeleton sequences.
HumanML3D = the final processed dataset (joint sequences + text).
If you want, I can draw a simple “data flow” diagram of how AMASS → HumanML3D works.

Here’s a simple data‑flow view:

AMASS (many MoCap datasets)
   |
   |  (SMPL+H + DMPL decode pose parameters)
   v
raw joint sequences  -->  pose_data/
   |
   |  (segment + normalize + mirror)
   v
motion clips  -->  joints/
   |
   |  (feature extraction)
   v
new_joints/ + new_joint_vecs/ + Mean.npy + Std.npy
   |
   |  (paired with text)
   v
HumanML3D dataset
AMASS gives pose parameters.
SMPL+H/DMPL turn those parameters into 3D joints.
pose_data/ is raw per‑sequence joints.
joints/ is segmented clips + mirrored augmentations.
new_joints/new_joint_vecs are the processed representations used by text‑to‑motion models.
Text captions are aligned by ID in texts/.

You just completed the first stage of the HumanML3D pipeline: converting AMASS + HumanAct12 motions into unified joint‑space clips and augmenting them.

What we did

Loaded AMASS .npz motions and converted them to joint sequences using SMPL+H + DMPL models.
Collected pose sequences into pose_data/ (raw per‑sequence poses).
Segmented long sequences into clips, normalized, and mirrored them.
Wrote out per‑clip joint sequences into joints/ (and mirrored versions with M prefix).
What folders/files were created or updated

HumanML3D/pose_data/
Contains converted pose sequences for each AMASS subset and HumanAct12 (e.g., pose_data/ACCAD/..., pose_data/humanact12/...).
HumanML3D/joints/
Final segmented joint clips as .npy files (e.g., 000001.npy, M000001.npy).
M prefix files are mirrored motions.
index.csv
The index file used for segmentation; links each output clip to a source path and frame range.
Next steps are:

Run motion_representation.ipynb to convert joints/ into motion features (new_joints/, new_joint_vecs/).
Run cal_mean_variance.ipynb to produce Mean.npy and Std.npy.